
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>

<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.5 - alpha</p>
<p>Package compatibility: squid-py v0.5.4, keeper-contracts 0.8.6, utilities 0.2.0,
<p>Component compatibility: Brizo v0.2.9, Aquarius v0.1.11, Nile testnet smart contracts 0.8.6</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

Getting Underway - Downloading Datasets (Assets)
To complete the basic datascience workflow, this notebook will demonstrate how a user
can download an asset. Downloading an asset is a simple example of a Service Execution Agreement -
similar to a contract with a series of clauses. Each clause is secured on the blockchain, allowing for trustful
execution of a contract.

In this notebook, an asset will be first published as before, and then ordered and downloaded.

### Section 0: Import modules, and setup logging

In [1]:
import logging
import os
from squid_py import Metadata, Ocean
import squid_py
import mantaray_utilities as manta_utils

# Setup logging
from mantaray_utilities.user import get_account_from_config
from mantaray_utilities.blockchain import subscribe_event
manta_utils.logging.logger.setLevel('INFO')
import mantaray_utilities as manta_utils
from squid_py import Config
from squid_py.keeper import Keeper
from pathlib import Path
import datetime
import web3

10 - logging         - <module>        - Logging started


Using default logging settings.


In [2]:
print("Mantaray utilities==",manta_utils.__version__)
print("squid-py==",  squid_py.__version__)

Mantaray utilities== 0.2.1
squid-py== 0.5.11


## Section 1: Get the configuration from the INI file

In [3]:
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))
config_from_ini = Config(CONFIG_INI_PATH)

50 - <ipython-input-3-a85e5e88e978> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-3-a85e5e88e978> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini
50 - <ipython-input-3-a85e5e88e978> - <module>        - Squid API version: 0.5.11


## Section 2: Delegate access of your asset to the marketplace
When we publish a register a DDO to a marketplace, we assign several services and conditions on those services.
By default, the permission to grant access will lie with you, the publisher. As a publisher, you would need to
run the services component (brizo), in order to manage access to your assets.

However, for the case of a marketplace, we will delegate permission to grant access to these services to the market
place on our behalf. Therefore, we will need the public address of the marketplace component. Of course, the
conditions are defined ultimately by you, the publisher. In the test-net, the provider has the following address:

In [4]:
MARKET_PLACE_PROVIDER_ADDRESS = web3.Web3.toChecksumAddress('0x376817c638d2a04f475a73af37f7b51a2862d567')

## Section 3: Instantiate Ocean

In [5]:
ocn = Ocean(config_from_ini)
keeper = Keeper.get_instance()

20 - ocean_secret_store - __init__        - 	SecretStore: url https://secret-store.dev-ocean.com, parity-client https://nile.dev-ocean.com, account 0x44E07D8190Cc0f0eCCC9C935D5EA47194301Cb00
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: /home/batman/ocn/mantaray_jupyter/artifacts_nile
20 - diagnostics     - verify_contracts - Using keeper contracts from network nile, network id is 8995
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".nile.json", e.g. "AgreementStoreManager.nile.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0x865396b7ddc58C693db7FCAD1168E3BD95Fe3368
	OceanToken: 0x9861Da395d7da984D5E8C712c2EDE44b41F777Ad
	DIDRegistry: 0x4A0f7F763B1A7937aED21D63b2A78adc89c5Db23
	AgreementStoreManager: 0x62f84700b1A0ea6Bfb505aDC3c0286B7944D247C
	TemplateStoreManager: 0x9768c8ae44f1dc81cAA98F48792aA5730cAd2F73
	ConditionStoreManager: 0x39b0AA775496C5ebf26f3B81C9e

## Section 4: Get Publisher and register an asset for testing the download
Of course, you can download your own asset, one that you have created, or
one that you have found via the search api. All you need is the DID of the asset.

In [6]:
publisher_account = manta_utils.user.get_account_by_index(ocn,0)

# publisher_account = get_account_from_config(config_from_ini, 'parity.address', 'parity.password')
print("Publisher address: {}".format(publisher_account.address))
print("Publisher   ETH: {:0.1f}".format(ocn.accounts.balance(publisher_account).eth/10**18))
print("Publisher OCEAN: {:0.1f}".format(ocn.accounts.balance(publisher_account).ocn/10**18))

20 - user            - load_passwords  - 5 account-password pairs loaded


Publisher address: 0x0d4230b1795959ab5453eb98E4853966bcD3059b
Publisher   ETH: 111.0
Publisher OCEAN: 3.0


In [7]:
# Register an asset
ddo = ocn.assets.create(Metadata.get_example(), publisher_account, providers=[MARKET_PLACE_PROVIDER_ADDRESS])
logging.info(f'registered ddo: {ddo.did}')

20 - ocean_assets    - create          - Content urls encrypted successfully 0x0a28229e6c4cb19966463f4e77407fd42fea6bd198853727ba3eea11a62992ecb3fd4dc3f3a6fb77d1385df3de537b56119f0fe42665fef4e7fc274119483380db7ff40cdf9a66458b945592f7ba3256b18ce51dfbdbc9477feb2a783671079312b1f9afa0e0e168c5e2ec2256e108083638f3a98a85aae7e6636badbf31adcd816396ccf45876ef329f0b2d829d3bfe3a4836d360158d05567493449dd342714d3ecb98e7ff80502bcf1a7974e3003d8d8b2d46324bf1625a947eff319de7329ef6bd1c1d226cf2b579eb50eed00ee3dd737ea5092d77ca2d068c7049765061a1df6c8e7a1177d7a87232c927542ea51e1126b86b23d9adae73d2673e1e85ba7816ecc6d300a750ffef78098b0cc84a7baa2251c049f131e0988bc5472d7913f40d1871b33f4e73c099e5298c09365af92a4b9abd1372c4f864a513cbe48807322e610eec1a6fdaa357aa4ab6ce8a1f3b269c30b3103161a14db76dd93ef6169021468a407f492352dec09e9839b0347f263b118936e6dbfab61bcacdb27813d389102d4876919040da25ffa2b4fd966a8fca9f0c93d46d07c6f8b2cc9fffe9a35ce53164e37cc43186a779df1a0fd211d6cf7e60512d274865eb81e2ff660997943afa7159fc374f67ef272

## Section 5: Get Consumer account, ensure token balance

In [8]:
# consumer_account = get_account_from_config(config_from_ini, 'parity.address1', 'parity.password1')
consumer_account = manta_utils.user.get_account_by_index(ocn,1)
print("Consumer address: {}".format(consumer_account.address))
print("Consumer   ETH: {:0.1f}".format(ocn.accounts.balance(consumer_account).eth/10**18))
print("Consumer OCEAN: {:0.1f}".format(ocn.accounts.balance(consumer_account).ocn/10**18))
assert ocn.accounts.balance(consumer_account).eth/10**18 > 1, "Insuffient ETH in account {}".format(consumer_account.address)
# Ensure the consumer always has 10 OCEAN
if ocn.accounts.balance(consumer_account).ocn/10**18 < 10:
    refill_amount = int(10 - ocn.accounts.balance(consumer_account).ocn/10**18)
    logging.info("Requesting {} tokens".format(refill_amount))
    ocn.accounts.request_tokens(consumer_account, refill_amount)

20 - user            - load_passwords  - 5 account-password pairs loaded


Consumer address: 0x0ed597b76592b4E63F0494127ADFB7Ea0829e337
Consumer   ETH: 100.0
Consumer OCEAN: 10.0


## Section 6: Initiate the agreement for accessing (downloading) the asset, wait for condition events

In [9]:
agreement_id = ocn.assets.order(ddo.did, 'Access', consumer_account)
logging.info("Consumer has placed an order for asset {}".format(ddo.did))
logging.info("The service agreement ID is {}".format(agreement_id))

20 - access_secret_store_template - subscribe_agreement_created - Subscribing AgreementCreated event with agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339.
20 - brizo           - initialize_service_agreement - Service agreement initialized successfully, service agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339, purchaseEndpoint https://nginx-brizo.dev-ocean.com//api/v1/brizo/services/access/initialize
20 - <ipython-input-9-f0fa9af99598> - <module>        - Consumer has placed an order for asset did:op:7817a0c233904e19b0e8ab030ff1171adc2b64b3caf245f6b2c3b09bcfc1cea1
20 - <ipython-input-9-f0fa9af99598> - <module>        - The service agreement ID is 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339


In Ocean Protocol, downloading an asset is enforced by a contract.
The contract conditions and clauses are set by the publisher. Conditions trigger events, which are monitored
to ensure the contract is successfully executed.

In [10]:
# Listen to events in the download process
subscribe_event("created agreement", keeper, agreement_id)
subscribe_event("lock reward", keeper, agreement_id)
subscribe_event("access secret store", keeper, agreement_id)
subscribe_event("escrow reward", keeper, agreement_id)

20 - access_secret_store_template - subscribe_agreement_created - Subscribing AgreementCreated event with agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339.
20 - condition_base  - subscribe_condition_fulfilled - Subscribing Fulfilled event with agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339.


Received event AgreementCreated.


20 - condition_base  - subscribe_condition_fulfilled - Subscribing Fulfilled event with agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339.


Received event Fulfilled.


20 - condition_base  - subscribe_condition_fulfilled - Subscribing Fulfilled event with agreement id 0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339.


Received event Fulfilled.
Received event Fulfilled.


Now that the agreement is signed, the consumer can download the asset.

In [11]:
assert ocn.agreements.is_access_granted(agreement_id, ddo.did, consumer_account.address)

ocn.assets.consume(agreement_id, ddo.did, 'Access', consumer_account, 'downloads_nile')
logging.info('Success buying asset.')

20 - brizo           - consume_service - invoke consume endpoint with this url: https://nginx-brizo.dev-ocean.com//api/v1/brizo/services/consume?url=https://testocnfiles.blob.core.windows.net/testfiles/testzkp.pdf&serviceAgreementId=0x36f2bbb0391b4caabc60d5d0427e82816fb7d985b45f49e2a6041b45c1abd339&consumerAddress=0x0ed597b76592b4E63F0494127ADFB7Ea0829e337
20 - utils           - process_tx_receipt - Waiting for transaction receipt timed out. Cannot verify receipt and event.
20 - brizo           - consume_service - Total size of testzkp.pdf: 0 bytes.
497796KB [00:02, 232744.74KB/s]20 - brizo           - consume_service - Saved downloaded file in /home/batman/ocn/mantaray_jupyter/downloads_nile/datafile.7817a0c233904e19b0e8ab030ff1171adc2b64b3caf245f6b2c3b09bcfc1cea1.1/testzkp.pdf
20 - brizo           - consume_service - invoke consume endpoint with this url: https://nginx-brizo.dev-ocean.com//api/v1/brizo/services/consume?url=s3://ocean-test-osmosis-data-plugin-dataseeding-1537375953/da